In [1]:
import cv2 
import numpy as np
import pickle
import matplotlib.pyplot as plt
import time

### Choose data

In [2]:
tinynet = 1
cifar = 2
data = cifar

In [22]:
# Function to unpickle CIFAR data
def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

if data == cifar :

    start_t = time.time()
    # Paths
    metadata_path = r'D:\Study\Uni_Projects\Machine Vision\Milestone-1\cifar-100\meta'  # Change this path
    data_pre_path = r"D:\Study\Uni_Projects\Machine Vision\Milestone-1\cifar-100\\"  # Change this path
    data_train_path = data_pre_path + 'train'
    data_test_path = data_pre_path + 'test'

    # Load metadata and superclass labels
    metadata = unpickle(metadata_path)
    superclass_dict = dict(enumerate(metadata[b'coarse_label_names']))

    # Load training and testing data
    data_train_dict = unpickle(data_train_path)
    data_test_dict = unpickle(data_test_path)

    # Extract training data
    cv_images = data_train_dict[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # Reshape to (N, H, W, C)
    labels = np.array(data_train_dict[b'fine_labels'])  # Use fine labels for 100 classes


    # cv_images_test = data_test_dict[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    # labels_test = np.array(data_test_dict[b'fine_labels'])


    print(f"Data Loading Time: {time.time() - start_t:.2f} seconds")


    # Shape of data
    print(f"Training images shape: {cv_images.shape}")
    print(f"Training labels shape: {labels.shape}")
    # # Print shapes of test data
    # print(f"Test images shape: {cv_images_test.shape}")
    # print(f"Test labels shape: {labels_test.shape}")



Data Loading Time: 0.16 seconds
Training images shape: (50000, 32, 32, 3)
Training labels shape: (50000,)


# Data preprocessing (only once)

In [4]:
# contains available classes in this data

with open(r"D:\Study\Uni_Projects\Machine Vision\Milestone-1\wnids.txt" , "r") as f:
    classes = []
    for line in f :
        line = line.removesuffix("\n")
        classes.append(line)



# contains all classes in old file
with open(r"D:\Study\Uni_Projects\Machine Vision\Milestone-1\words.txt" , "r") as f:
    labels =[]
    for line in f :
        line = line.removesuffix("\n")
        line = line.split("\t")
        
        labels.append(line)
        


### make file with names of classes in subset (only once)

In [5]:
existing = [] 
for file , label in labels:
    if file in classes:
        existing.append(f"{file}-{label}\n")
with open(r"D:\Study\Uni_Projects\Machine Vision\Milestone-1\tiny_file-class","+w") as f:
    f.writelines(existing[0:100])

# Image import 

In [6]:
files_cls = []
with open(r"D:\Study\Uni_Projects\Machine Vision\Milestone-1\tiny_file-class" , "r") as f :
    for line in f :
        line = line.strip()
        
        file_name , labels = line.split("-", maxsplit=1)
        labels = labels.split(",")
        files_cls.append([file_name, labels])


In [7]:
import os

train_dir = r"D:\Study\Uni_Projects\Machine Vision\Milestone-1\train-100"
# uses second half


image_data = []

for root, dirs, files in os.walk(train_dir):
    for file in files:
        if file.endswith((".jpg", ".JPEG", ".png")):
            file_path = os.path.join(root, file)
 
            label = os.path.basename(os.path.dirname(root))
            
            image_data.append((file_path, label))
            

# Print some of the data

print("Number of images:", len(image_data))

file_path # path to each image
label # label code of each image


Number of images: 50000


'n03250847'

In [8]:
if data == tinynet :
    cv_images = []
    labels = []
    # make it np
    i = 1
    num = len(image_data)
    start_t = time.time()

    for name , cls in image_data:
        image = cv2.imread(name)
        labels.append(cls)
        cv_images.append(image)
        
        if i%200 == 0:
            print(f"{i}/{num}")

        i=i+1
        
    cv_images = np.array(cv_images)
    labels = np.array(labels)


    print(time.time()-start_t)

##### labels

# feature extraction

In [10]:
from skimage.feature import hog , local_binary_pattern

# HOG 
def extract_hog(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features, _ = hog(gray_image, orientations=9, pixels_per_cell=(8,8),
                      cells_per_block=(2, 2), block_norm='L2-Hys',
                      visualize=True, feature_vector=True)
    return features

# LBP 
def extract_lbp(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    radius = 3
    n_points = 8 * radius
    lbp = local_binary_pattern(gray_image, n_points, radius, method="uniform")
    
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), density=True)
    return lbp_hist

# Color 

def extract_color_hist(image):
    hist_red = cv2.calcHist([image], [0], None, [32], [0, 256])
    hist_green = cv2.calcHist([image], [1], None, [32], [0, 256])
    hist_blue = cv2.calcHist([image], [2], None, [32], [0, 256])
    color_hist = np.concatenate([hist_red, hist_green, hist_blue]).flatten()
    return color_hist / color_hist.sum()  # Normalize


# reduce size for trial

# cv_images = cv_images[:10000]
# labels = labels[:10]

hog_features = []
lbp_features = []
color_hist_features = []

lbp_features64 = []
color_hist_features64 = []

hog_features32 = []
lbp_features32 = []
color_hist_features32 = []

start_t = time.time()


i = 1

for image in cv_images:
    if image is None:  # Skip invalid images
        continue
    # Resize the image to a fixed size for consistency
    image128 = cv2.resize(image, (128, 128))
    image64 = cv2.resize(image, (64,64))
    image32 = cv2.resize(image, (32, 32))         # For 32x32 features
    
    # using different sizes
    hog_feat = extract_hog(image64)
    lbp_feat64 = extract_lbp(image64)
    color_feat64 = extract_color_hist(image64)
    
    
    lbp_feat = extract_lbp(image128)
    color_feat = extract_color_hist(image128)
    
    #################################################added 
    hog_feat32 = extract_hog(image32)
    lbp_feat32 = extract_lbp(image32)
    color_feat32 = extract_color_hist(image32)
    
    # Append to respective lists
    hog_features.append(hog_feat)
    lbp_features.append(lbp_feat)
    color_hist_features.append(color_feat)
    
    lbp_features64.append(lbp_feat64)
    color_hist_features64.append(color_feat64)

    hog_features32.append(hog_feat32)
    lbp_features32.append(lbp_feat32)
    color_hist_features32.append(color_feat32)    
    
    
    if i % 400 == 0:
        print(f"Processed {i}/50000 of images")
    i += 1

# Convert lists to NumPy arrays for further processing
hog_features = np.array(hog_features)
lbp_features = np.array(lbp_features)
color_hist_features = np.array(color_hist_features)

lbp_features64 = np.array(lbp_features64)
color_hist_features64 = np.array(color_hist_features64)

hog_features32 = np.array(hog_features32)
lbp_features32 = np.array(lbp_features32)
color_hist_features32 = np.array(color_hist_features32)


print(f"HOG features: {hog_features.shape}")
print(f"LBP features: {lbp_features.shape}")
print(f"Color Histogram features: {color_hist_features.shape}")
print(f"LBP 64 features: {lbp_features64.shape}")
print(f"Color Histogram 64 features: {color_hist_features64.shape}")
print(f"HOG 32 features: {hog_features32.shape}")
print(f"LBP 32 features: {lbp_features32.shape}")
print(f"Color Histogram 32 features: {color_hist_features32.shape}")

print(f"{time.time()-start_t} sec")
extraction_time = time.time()-start_t

Processed 400/50000 of images
Processed 800/50000 of images
Processed 1200/50000 of images
Processed 1600/50000 of images
Processed 2000/50000 of images
Processed 2400/50000 of images
Processed 2800/50000 of images
Processed 3200/50000 of images
Processed 3600/50000 of images
Processed 4000/50000 of images
Processed 4400/50000 of images
Processed 4800/50000 of images
Processed 5200/50000 of images
Processed 5600/50000 of images
Processed 6000/50000 of images
Processed 6400/50000 of images
Processed 6800/50000 of images
Processed 7200/50000 of images
Processed 7600/50000 of images
Processed 8000/50000 of images
Processed 8400/50000 of images
Processed 8800/50000 of images
Processed 9200/50000 of images
Processed 9600/50000 of images
Processed 10000/50000 of images
Processed 10400/50000 of images
Processed 10800/50000 of images
Processed 11200/50000 of images
Processed 11600/50000 of images
Processed 12000/50000 of images
Processed 12400/50000 of images
Processed 12800/50000 of images
Pr

In [17]:


print(f"HOG features: {hog_features.shape}")
print(f"LBP features: {lbp_features.shape}")
print(f"Color Histogram features: {color_hist_features.shape}")
print(f"LBP 64 features: {lbp_features64.shape}")
print(f"Color Histogram 64 features: {color_hist_features64.shape}")
print(f"HOG 32 features: {hog_features32.shape}")
print(f"LBP 32 features: {lbp_features32.shape}")
print(f"Color Histogram 32 features: {color_hist_features32.shape}")


HOG features: (50000, 1764)
LBP features: (50000, 26)
Color Histogram features: (50000, 96)
LBP 64 features: (50000, 26)
Color Histogram 64 features: (50000, 96)
HOG 32 features: (50000, 324)
LBP 32 features: (50000, 26)
Color Histogram 32 features: (50000, 96)


# classifiers

In [12]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import Counter

def kmeans_clustering(features, num_clusters, max_iters=30, tol=1e-4, show = True):
    np.random.seed(40)
    centers = features[np.random.choice(features.shape[0], num_clusters, replace=False)]
    
    for i in range(max_iters):
        distances = np.linalg.norm(features[:, np.newaxis] - centers, axis=2)
        cluster_labels = np.argmin(distances, axis=1)
        
        new_centers = np.array([features[cluster_labels == k].mean(axis=0) for k in range(num_clusters)])
        if np.linalg.norm(new_centers - centers) < tol:
            break
        centers = new_centers
        if (show == True):
            print(f"step {i}")
    return cluster_labels, centers



def knn_predict(train_features, train_labels, test_features, k=10):
    predictions = []
    num = len(test_features)
    batch_size = 50  # Process test samples in batches for better memory utilization
    num_batches = (num + batch_size - 1) // batch_size

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, num)
        test_batch = test_features[start_idx:end_idx]
        
        distances = np.linalg.norm(train_features[:, None, :] - test_batch[None, :, :], axis=2)  # Shape: (train_size, batch_size)
        
        # Find nearest neighbors
        nearest_indices = np.argsort(distances, axis=0)[:k, :] 
        nearest_labels = train_labels[nearest_indices]  # Get the labels of the nearest neighbors
        
        # Determine the most common label for each test sample
        batch_predictions = [Counter(nearest_labels[:, col]).most_common(1)[0][0] for col in range(test_batch.shape[0])]
        predictions.extend(batch_predictions)
        
        print(f"Batch {batch_idx + 1}/{num_batches} completed")

    return np.array(predictions)



def train_svm(train_features, train_labels):
    svm = SVC(kernel="linear", C=1.0)
    svm.fit(train_features, train_labels)
    return svm




# train

In [23]:

hog_features_array = np.array(hog_features)
lbp_features_array = np.array(lbp_features)
color_hist_features_array = np.array(color_hist_features)

lbp_features_array64 = np.array(lbp_features64)
color_hist_features_array64 = np.array(color_hist_features64)

# 32x32 features
hog_features_array32 = np.array(hog_features32)
lbp_features_array32 = np.array(lbp_features32)
color_hist_features_array32 = np.array(color_hist_features32)

labels_array = np.array(labels)

# Split Data into Training and Testing
X_train_hog, X_test_hog, y_train, y_test = train_test_split(hog_features_array, labels_array, test_size=0.2, random_state=42)
X_train_lbp, X_test_lbp, y_train_lbp, y_test_lbp= train_test_split(lbp_features_array, labels_array, test_size=0.2, random_state=42)
X_train_color, X_test_color, y_train_color , y_test_color = train_test_split(color_hist_features_array, labels_array, test_size=0.2, random_state=42)

X_train_lbp64, X_test_lbp64, y_train_lbp64, y_test_lbp64= train_test_split(lbp_features_array64, labels_array, test_size=0.2, random_state=42)
X_train_color64, X_test_color64, y_train_color64 , y_test_color64 = train_test_split(color_hist_features_array64, labels_array, test_size=0.2, random_state=42)


X_train_hog32, X_test_hog32, y_train_hog32, y_test_hog32 = train_test_split(hog_features_array32, labels_array, test_size=0.2, random_state=42)
X_train_lbp32, X_test_lbp32, y_train_lbp32, y_test_lbp32 = train_test_split(lbp_features_array32, labels_array, test_size=0.2, random_state=42)
X_train_color32, X_test_color32, y_train_color32, y_test_color32 = train_test_split(color_hist_features_array32, labels_array, test_size=0.2, random_state=42)





### HOG

In [26]:
print("Training SVM...")
start_time = time.time()
svm_model_hog = train_svm(X_train_hog, y_train)
svm_predictions_hog = svm_model_hog.predict(X_test_hog)
svm_accuracy_hog = accuracy_score(y_test, svm_predictions_hog)
print(f"SVM (HOG) Accuracy: {svm_accuracy_hog}, takiing: {time.time()-start_time} sec")
svm_hog_time = time.time() - start_time

Training SVM...
SVM (HOG) Accuracy: 0.2088, takiing: 1289.175202846527 sec


### Color hist

In [ ]:


#  KNN
print("Training K-Nearest Neighbors for color...")
start_time = time.time()
knn_predictions_color = knn_predict(X_train_color, y_train_color, X_test_color, k=10)
knn_accuracy_color = accuracy_score(y_test_color, knn_predictions_color)
print(f"K-Nearest Neighbors (color) Accuracy: {knn_accuracy_color}, taking: {time.time() - start_time} sec")
knn_color_time = time.time() - start_time

# SVM
print("Training SVM for color...")
start_time = time.time()
svm_model_color = train_svm(X_train_color, y_train_color)
svm_predictions_color = svm_model_color.predict(X_test_color)
svm_accuracy_color= accuracy_score(y_test_color, svm_predictions_color)
print(f"SVM (color) Accuracy: {svm_accuracy_color}, taking: {time.time() - start_time} sec")
svm_color_time = time.time() - start_time


Training K-Nearest Neighbors for color...
Batch 1/200 completed
Batch 2/200 completed
Batch 3/200 completed
Batch 4/200 completed
Batch 5/200 completed
Batch 6/200 completed
Batch 7/200 completed
Batch 8/200 completed
Batch 9/200 completed
Batch 10/200 completed
Batch 11/200 completed
Batch 12/200 completed
Batch 13/200 completed
Batch 14/200 completed
Batch 15/200 completed
Batch 16/200 completed
Batch 17/200 completed
Batch 18/200 completed
Batch 19/200 completed
Batch 20/200 completed
Batch 21/200 completed
Batch 22/200 completed
Batch 23/200 completed
Batch 24/200 completed
Batch 25/200 completed
Batch 26/200 completed
Batch 27/200 completed
Batch 28/200 completed
Batch 29/200 completed
Batch 30/200 completed
Batch 31/200 completed
Batch 32/200 completed
Batch 33/200 completed
Batch 34/200 completed
Batch 35/200 completed
Batch 36/200 completed
Batch 37/200 completed
Batch 38/200 completed
Batch 39/200 completed
Batch 40/200 completed
Batch 41/200 completed
Batch 42/200 completed
B

### kmean with color


In [ ]:
# kmean
def kmeans_clustering(features, num_clusters, max_iters=30, tol=1e-4):

    np.random.seed(40)
    centers = features[np.random.choice(features.shape[0], num_clusters, replace=False)]
    
    for i in range(max_iters):
        # Calculate distances and assign points to the nearest center
        distances = np.linalg.norm(features[:, np.newaxis] - centers, axis=2)
        cluster_labels = np.argmin(distances, axis=1)
        
        # Update centers
        new_centers = np.array([features[cluster_labels == k].mean(axis=0) for k in range(num_clusters)])
        if np.linalg.norm(new_centers - centers) < tol:
            break
        centers = new_centers
        
        print(f"step {i+1}")
    
    return centers


def assign_labels_to_clusters(centers, features, labels, k=500):

    cluster_labels = {}
    
    for idx, center in enumerate(centers):
        # Compute distances to all features
        distances = np.linalg.norm(features - center, axis=1)
        nearest_indices = np.argsort(distances)[:k]
        nearest_labels = labels[nearest_indices]

        most_common_label = Counter(nearest_labels).most_common(1)[0][0]
        cluster_labels[idx] = most_common_label
    
    return cluster_labels


def predict_using_kmeans(centers, cluster_labels, test_features):

    predictions = []
    
    for test_sample in test_features:
        # Find the nearest center
        distances = np.linalg.norm(centers - test_sample, axis=1)
        nearest_center_idx = np.argmin(distances)
        
        # Use the label of the nearest center
        predictions.append(cluster_labels[nearest_center_idx])
    
    return np.array(predictions)


num_clusters = len(np.unique(labels_array))
start_time = time.time()
# Train K-Means without labels
centers = kmeans_clustering(X_train_color, num_clusters)
cluster_labels = assign_labels_to_clusters(centers, X_train_color, y_train_color, k=500)
kmeans_predictions = predict_using_kmeans(centers, cluster_labels, X_test_color)
kmeans_accuracy = accuracy_score(y_test_color, kmeans_predictions)
print(f"K-Means Accuracy: {kmeans_accuracy}")

kmeans_color_time = time.time() - start_time


step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9
step 10
step 11
step 12
step 13
step 14
step 15
step 16
step 17
step 18
step 19
step 20
step 21
step 22
step 23
step 24
step 25
step 26
step 27
step 28
step 29
step 30
K-Means Accuracy: 0.0467


### LBP

In [ ]:

# K-Nearest Neighbors
print("Training K-Nearest Neighbors for LBP...")
start_time = time.time()
knn_predictions_lbp = knn_predict(X_train_lbp, y_train_lbp, X_test_lbp, k=10)
knn_accuracy_lbp = accuracy_score(y_test_lbp, knn_predictions_lbp)
print(f"K-Nearest Neighbors (LBP) Accuracy: {knn_accuracy_lbp}, taking: {time.time() - start_time} sec")
knn_lbp_time = time.time() - start_time
# Support Vector Machine (SVM)
print("Training SVM for LBP...")
start_time = time.time()
svm_model_lbp = train_svm(X_train_lbp, y_train_lbp)
svm_predictions_lbp = svm_model_lbp.predict(X_test_lbp)
svm_accuracy_lbp = accuracy_score(y_test_lbp, svm_predictions_lbp)
print(f"SVM (LBP) Accuracy: {svm_accuracy_lbp}, taking: {time.time() - start_time} sec")
svm_lbp_time = time.time() - start_time


Training K-Nearest Neighbors for LBP...
Batch 1/200 completed
Batch 2/200 completed
Batch 3/200 completed
Batch 4/200 completed
Batch 5/200 completed
Batch 6/200 completed
Batch 7/200 completed
Batch 8/200 completed
Batch 9/200 completed
Batch 10/200 completed
Batch 11/200 completed
Batch 12/200 completed
Batch 13/200 completed
Batch 14/200 completed
Batch 15/200 completed
Batch 16/200 completed
Batch 17/200 completed
Batch 18/200 completed
Batch 19/200 completed
Batch 20/200 completed
Batch 21/200 completed
Batch 22/200 completed
Batch 23/200 completed
Batch 24/200 completed
Batch 25/200 completed
Batch 26/200 completed
Batch 27/200 completed
Batch 28/200 completed
Batch 29/200 completed
Batch 30/200 completed
Batch 31/200 completed
Batch 32/200 completed
Batch 33/200 completed
Batch 34/200 completed
Batch 35/200 completed
Batch 36/200 completed
Batch 37/200 completed
Batch 38/200 completed
Batch 39/200 completed
Batch 40/200 completed
Batch 41/200 completed
Batch 42/200 completed
Bat

#### LBP kmean

In [ ]:

# Train K-Means without labels
start_time = time.time()
centers = kmeans_clustering(X_train_lbp, num_clusters)
cluster_labels = assign_labels_to_clusters(centers, X_train_lbp, y_train_lbp, k=500)
kmeans_predictions = predict_using_kmeans(centers, cluster_labels, X_test_lbp)
kmeans_accuracy_lbp = accuracy_score(y_test_lbp, kmeans_predictions)
print(f"K-Means Accuracy: {kmeans_accuracy_lbp}")

kmeans_lbp_time = time.time() - start_time

step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9
step 10
step 11
step 12
step 13
step 14
step 15
step 16
step 17
step 18
step 19
step 20
step 21
step 22
step 23
step 24
step 25
step 26
step 27
step 28
step 29
step 30
K-Means Accuracy: 0.0508


### image 64x64 size

In [ ]:
# KNN for 64x64 LBP Features
print("Training K-Nearest Neighbors for LBP (64x64)...")
start_time = time.time()
knn_predictions_lbp64 = knn_predict(X_train_lbp64, y_train_lbp64, X_test_lbp64, k=10)
knn_accuracy_lbp64 = accuracy_score(y_test_lbp64, knn_predictions_lbp64)
print(f"K-Nearest Neighbors (LBP 64x64) Accuracy: {knn_accuracy_lbp64}, taking: {time.time() - start_time:.2f} sec")
knn_64_lbp_time = time.time() - start_time

# SVM for 64x64 LBP Features
print("Training SVM for LBP (64x64)...")
start_time = time.time()
svm_model_lbp64 = train_svm(X_train_lbp64, y_train_lbp64)
svm_predictions_lbp64 = svm_model_lbp64.predict(X_test_lbp64)
svm_accuracy_lbp64 = accuracy_score(y_test_lbp64, svm_predictions_lbp64)
print(f"SVM (LBP 64x64) Accuracy: {svm_accuracy_lbp64}, taking: {time.time() - start_time:.2f} sec")
svm_64_lbp_time = time.time() - start_time

# KNN for 64x64 Color Features
print("Training K-Nearest Neighbors for Color (64x64)...")
start_time = time.time()
knn_predictions_color64 = knn_predict(X_train_color64, y_train_color64, X_test_color64, k=10)
knn_accuracy_color64 = accuracy_score(y_test_color64, knn_predictions_color64)
print(f"K-Nearest Neighbors (Color 64x64) Accuracy: {knn_accuracy_color64}, taking: {time.time() - start_time:.2f} sec")
knn_64_color_time = time.time() - start_time

# SVM for 64x64 Color Features
print("Training SVM for Color (64x64)...")
start_time = time.time()
svm_model_color64 = train_svm(X_train_color64, y_train_color64)
svm_predictions_color64 = svm_model_color64.predict(X_test_color64)
svm_accuracy_color64 = accuracy_score(y_test_color64, svm_predictions_color64)
print(f"SVM (Color 64x64) Accuracy: {svm_accuracy_color64}, taking: {time.time() - start_time:.2f} sec")
svm_64_color_time = time.time() - start_time


Training K-Nearest Neighbors for LBP (64x64)...
Batch 1/200 completed
Batch 2/200 completed
Batch 3/200 completed
Batch 4/200 completed
Batch 5/200 completed
Batch 6/200 completed
Batch 7/200 completed
Batch 8/200 completed
Batch 9/200 completed
Batch 10/200 completed
Batch 11/200 completed
Batch 12/200 completed
Batch 13/200 completed
Batch 14/200 completed
Batch 15/200 completed
Batch 16/200 completed
Batch 17/200 completed
Batch 18/200 completed
Batch 19/200 completed
Batch 20/200 completed
Batch 21/200 completed
Batch 22/200 completed
Batch 23/200 completed
Batch 24/200 completed
Batch 25/200 completed
Batch 26/200 completed
Batch 27/200 completed
Batch 28/200 completed
Batch 29/200 completed
Batch 30/200 completed
Batch 31/200 completed
Batch 32/200 completed
Batch 33/200 completed
Batch 34/200 completed
Batch 35/200 completed
Batch 36/200 completed
Batch 37/200 completed
Batch 38/200 completed
Batch 39/200 completed
Batch 40/200 completed
Batch 41/200 completed
Batch 42/200 compl

### image 32x32

In [24]:
# Training SVM for 32x32 HOG Features
print("Training SVM for HOG (32x32)...")
start_time = time.time()
svm_model_hog32 = train_svm(X_train_hog32, y_train)
svm_predictions_hog32 = svm_model_hog32.predict(X_test_hog32)
svm_accuracy_hog32 = accuracy_score(y_test, svm_predictions_hog32)
print(f"SVM (HOG 32x32) Accuracy: {svm_accuracy_hog32}, taking: {time.time() - start_time:.2f} sec")
svm_32_hog_time = time.time() - start_time

# KNN for 32x32 LBP Features
print("Training K-Nearest Neighbors for LBP (32x32)...")
start_time = time.time()
knn_predictions_lbp32 = knn_predict(X_train_lbp32, y_train_lbp32, X_test_lbp32, k=10)
knn_accuracy_lbp32 = accuracy_score(y_test_lbp32, knn_predictions_lbp32)
print(f"K-Nearest Neighbors (LBP 32x32) Accuracy: {knn_accuracy_lbp32}, taking: {time.time() - start_time:.2f} sec")
knn_32_lbp_time = time.time() - start_time

# SVM for 32x32 LBP Features
print("Training SVM for LBP (32x32)...")
start_time = time.time()
svm_model_lbp32 = train_svm(X_train_lbp32, y_train_lbp32)
svm_predictions_lbp32 = svm_model_lbp32.predict(X_test_lbp32)
svm_accuracy_lbp32 = accuracy_score(y_test_lbp32, svm_predictions_lbp32)
print(f"SVM (LBP 32x32) Accuracy: {svm_accuracy_lbp32}, taking: {time.time() - start_time:.2f} sec")
svm_32_lbp_time = time.time() - start_time

# KNN for 32x32 Color Features
print("Training K-Nearest Neighbors for Color (32x32)...")
start_time = time.time()
knn_predictions_color32 = knn_predict(X_train_color32, y_train_color32, X_test_color32, k=10)
knn_accuracy_color32 = accuracy_score(y_test_color32, knn_predictions_color32)
print(f"K-Nearest Neighbors (Color 32x32) Accuracy: {knn_accuracy_color32}, taking: {time.time() - start_time:.2f} sec")
knn_32_color_time = time.time() - start_time

# SVM for 32x32 Color Features
print("Training SVM for Color (32x32)...")
start_time = time.time()
svm_model_color32 = train_svm(X_train_color32, y_train_color32)
svm_predictions_color32 = svm_model_color32.predict(X_test_color32)
svm_accuracy_color32 = accuracy_score(y_test_color32, svm_predictions_color32)
print(f"SVM (Color 32x32) Accuracy: {svm_accuracy_color32}, taking: {time.time() - start_time:.2f} sec")
svm_32_color_time = time.time() - start_time

Training SVM for HOG (32x32)...
SVM (HOG 32x32) Accuracy: 0.2038, taking: 272.07 sec
Training K-Nearest Neighbors for LBP (32x32)...
Batch 1/200 completed
Batch 2/200 completed
Batch 3/200 completed
Batch 4/200 completed
Batch 5/200 completed
Batch 6/200 completed
Batch 7/200 completed
Batch 8/200 completed
Batch 9/200 completed
Batch 10/200 completed
Batch 11/200 completed
Batch 12/200 completed
Batch 13/200 completed
Batch 14/200 completed
Batch 15/200 completed
Batch 16/200 completed
Batch 17/200 completed
Batch 18/200 completed
Batch 19/200 completed
Batch 20/200 completed
Batch 21/200 completed
Batch 22/200 completed
Batch 23/200 completed
Batch 24/200 completed
Batch 25/200 completed
Batch 26/200 completed
Batch 27/200 completed
Batch 28/200 completed
Batch 29/200 completed
Batch 30/200 completed
Batch 31/200 completed
Batch 32/200 completed
Batch 33/200 completed
Batch 34/200 completed
Batch 35/200 completed
Batch 36/200 completed
Batch 37/200 completed
Batch 38/200 completed
Ba

In [ ]:

with open("results.txt", "a") as f:
    f.write("\n-----------------------\n")
    f.write(f"dataset: {data}\n")
    f.write(f"parameters -> max_iter = 30, extraction_time: {extraction_time} \n")
    f.write(f"svm_hog_64 acc: {svm_accuracy_hog * 100:.2f}% time: {svm_hog_time:.2f} sec\n")
    f.write(f"svm_lbp acc: {svm_accuracy_lbp * 100:.2f}% time: {svm_lbp_time:.2f} sec \n")
    f.write(f"knn_lbp acc: {knn_accuracy_lbp * 100:.2f}% time: {knn_lbp_time:.2f} sec\n")
    f.write(f"kmean_lbp acc: {kmeans_accuracy_lbp * 100:.2f}% time: {kmeans_lbp_time:.2f} sec\n")
    f.write(f"svm_color acc: {svm_accuracy_color * 100:.2f}% time: {svm_color_time:.2f} sec\n")
    f.write(f"knn_color acc: {knn_accuracy_color * 100:.2f}% time: {knn_color_time:.2f} sec \n")
    f.write(f"kmean_color acc: {kmeans_accuracy * 100:.2f}% time: {kmeans_color_time:.2f} sec \n")
    
    # Add 64x64 accuracies
    f.write(f"\n")
    f.write(f"svm_lbp_64 acc: {svm_accuracy_lbp64 * 100:.2f}% time: {svm_64_lbp_time:.2f} sec \n")
    f.write(f"knn_lbp_64 acc: {knn_accuracy_lbp64 * 100:.2f}% time: {knn_64_lbp_time:.2f} sec\n")
    
    f.write(f"\n")
    f.write(f"svm_color_64 acc: {svm_accuracy_color64 * 100:.2f}% time: {svm_64_color_time:.2f} sec\n")
    f.write(f"knn_color_64 acc: {knn_accuracy_color64 * 100:.2f}% time: {knn_64_color_time:.2f} sec \n")
    
    # Add 32x32 accuracies
    f.write(f"\n")
    f.write(f"svm_hog_32 acc: {svm_accuracy_hog32 * 100:.2f}% time: {svm_32_hog_time:.2f} sec \n")
    
    f.write(f"\n")
    f.write(f"svm_lbp_32 acc: {svm_accuracy_lbp32 * 100:.2f}% time: {svm_32_lbp_time:.2f} sec \n")
    f.write(f"knn_lbp_32 acc: {knn_accuracy_lbp32 * 100:.2f}% time: {knn_32_lbp_time:.2f} sec\n")
    
    f.write(f"\n")
    f.write(f"svm_color_32 acc: {svm_accuracy_color32 * 100:.2f}% time: {svm_32_color_time:.2f} sec\n")
    f.write(f"knn_color_32 acc: {knn_accuracy_color32 * 100:.2f}% time: {knn_32_color_time:.2f} sec \n")
    
